In [1]:
from dotenv import load_dotenv  # Import the function to load environment variables
load_dotenv()  # Load environment variables from a .env file

import os
import openai 
from pinecone import Pinecone, ServerlessSpec


# Now you can use os.getenv() to get the variables


c:\Users\Jai Dayanand\anaconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
print(os.getenv("PINECONE_API_KEY"))


d2383add-ff4e-48e6-86e7-a96c7fa1ae82


In [6]:
# Initialize Pinecone client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create the index with a valid name
try:
    pc.create_index(
        name="new-rag-index",  # Valid name with lowercase letters and hyphens
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("Index 'new-rag-index' created successfully.")
except Exception as e:
    print(f"Error: {e}")


Index 'new-rag-index' created successfully.


In [7]:
import json 
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and her lab sessions are well-organized. She's always willing to help students outside of class hours."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Prof. Chen is an expert in his field and explains complex concepts clearly. His programming assignments are challenging but rewarding.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Psychology',
  'stars': 3,
  'review': "Dr. Thompson's research is fascinating, but her lectures can be a bit dry. She does provide good study materials though."},
 {'professor': 'Prof. David Martinez',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Prof. Martinez brings real-world examples into his lectures, making the subject more relatable. His exams are fair but challenging.'},
 {'professor': 'Dr. Rachel Lee',
  'subject': 'Chemistry',
  'stars': 5,
  'review': "Dr. Lee'

In [8]:
processed_data = []
client = openai

for reviews in data['reviews']:
    response = client.embeddings.create(
        input = reviews['review'],
        model="text-embedding-3-small",
    )
    embedding=response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":reviews['professor'],
        "metadata":{
            "review":reviews["review"],
            "subject": reviews["subject"],
            "stars":reviews["stars"]
        } 
        
    })

In [9]:
processed_data[0]

{'values': [-0.0057174027897417545,
  -0.015846865251660347,
  0.051769908517599106,
  0.03978685662150383,
  0.005332326516509056,
  0.0029011250007897615,
  0.013719156384468079,
  0.035061515867710114,
  -0.017556864768266678,
  0.011271637864410877,
  0.032424721866846085,
  -0.00930709671229124,
  -0.009874920360744,
  -0.0007648494793102145,
  0.0003465278132352978,
  0.04054395854473114,
  -0.02939632721245289,
  -0.009568165056407452,
  0.011297744698822498,
  0.06631142646074295,
  0.0328163243830204,
  -0.012674881145358086,
  0.024514343589544296,
  -0.05477219820022583,
  -0.020441673696041107,
  -0.06192547827959061,
  0.01328186597675085,
  0.026250449940562248,
  0.026250449940562248,
  -0.01567717082798481,
  0.07513555139303207,
  -0.011167210526764393,
  -0.041353270411491394,
  -0.013014270924031734,
  -0.028117090463638306,
  0.0439639575779438,
  0.030048998072743416,
  0.006826944649219513,
  0.018444499000906944,
  0.013334079645574093,
  -0.00573698291555047,
  

In [16]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [20]:
index.describe_index_stats() 

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}